In [2]:
pip install --upgrade bottleneck

  Obtaining dependency information for bottleneck from https://files.pythonhosted.org/packages/e1/ab/92c1292d7abcd424936f24470afc70a62601bd61bf95761832dfc88764da/Bottleneck-1.4.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/111.6 kB ? eta -:--:--
   ---------------------------------------- 111.6/111.6 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: bottleneck
    Found existing installation: Bottleneck 1.3.5
    Uninstalling Bottleneck-1.3.5:
      Successfully uninstalled Bottleneck-1.3.5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import subprocess
import os

# Function to run the download.py script with the appropriate parameters
def download_yt8m_shards(dataset_type='train', partition='2/frame', mirror='us', shards=''):
    """
    Downloads YouTube-8M dataset shards using the local download.py script.
    
    Args:
        dataset_type (str): Can be 'train', 'validate', or 'test'.
        partition (str): The partition path ('2/frame' for frame-level features).
        mirror (str): The mirror to use for downloading (e.g., 'us', 'eu', or 'asia').
        shards (str): Specifies the shard range to download (e.g., 'shard=1,1000').
    """
    
    # Set environment variables for partition and mirror
    os.environ['partition'] = partition + '/' + dataset_type
    os.environ['mirror'] = mirror
    
    # Adjust command to include the shard restriction (e.g., download the first 3 shards)
    base_command = f'python download.py --partition={partition}/{dataset_type} --mirror={mirror} {shards}'
    
    try:
        print(f"Starting download of {dataset_type} data... (Downloading {shards})")
        result = subprocess.run(base_command, shell=True, capture_output=True, text=True)
        
        # Print the standard output and error
        if result.stdout:
            print(f"Standard Output:\n{result.stdout}")
        if result.stderr:
            print(f"Standard Error:\n{result.stderr}")
        
        result.check_returncode()  # Check if the command was successful
        print(f"Successfully downloaded {dataset_type} data!")
    
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while downloading {dataset_type} data: {e}")
        print(f"Error Output: {e.stderr}")

# Set the directory where the download.py script is located
data_dir = os.path.expanduser("~/data/yt8m/frame")
os.makedirs(data_dir, exist_ok=True)
os.chdir(data_dir)

# Download 3 shards (around 1.2 GB total) for training, validation, and test sets
download_yt8m_shards(dataset_type='train', partition='2/frame', mirror='us', shards='shard=1,1000')
download_yt8m_shards(dataset_type='train', partition='2/frame', mirror='us', shards='shard=2,1000')
download_yt8m_shards(dataset_type='train', partition='2/frame', mirror='us', shards='shard=3,1000')

download_yt8m_shards(dataset_type='validate', partition='2/frame', mirror='us', shards='shard=1,1000')
download_yt8m_shards(dataset_type='test', partition='2/frame', mirror='us', shards='shard=1,1000')


Starting download of train data... (Downloading shard=1,1000)


In [ ]:
import subprocess
import os

# Function to run the download.py script with the appropriate parameters
def download_yt8m_shards(dataset_type='validate', partition='2/frame', mirror='us', shards=''):
    """
    Downloads YouTube-8M dataset shards using the local download.py script.
    
    Args:
        dataset_type (str): Can be 'train', 'validate', or 'test'.
        partition (str): The partition path ('2/frame' for frame-level features).
        mirror (str): The mirror to use for downloading (e.g., 'us', 'eu', or 'asia').
        shards (str): Specifies the shard range to download (e.g., 'shard=1,1000').
    """
    
    # Set environment variables for partition and mirror
    os.environ['partition'] = partition + '/' + dataset_type
    os.environ['mirror'] = mirror
    
    # Adjust command to include the shard restriction
    base_command = f'python download.py --partition={partition}/{dataset_type} --mirror={mirror} {shards}'
    
    try:
        print(f"Starting download of {dataset_type} data... (Downloading {shards})")
        result = subprocess.run(base_command, shell=True, capture_output=True, text=True)
        
        # Print the standard output and error
        if result.stdout:
            print(f"Standard Output:\n{result.stdout}")
        if result.stderr:
            print(f"Standard Error:\n{result.stderr}")
        
        result.check_returncode()  # Check if the command was successful
        print(f"Successfully downloaded {dataset_type} data!")
    
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while downloading {dataset_type} data: {e}")
        print(f"Error Output: {e.stderr}")

# Set the directory where the download.py script is located
data_dir = os.path.expanduser("~/data/yt8m/frame")
os.makedirs(data_dir, exist_ok=True)
os.chdir(data_dir)

# Download 3 shards for validation data
download_yt8m_shards(dataset_type='test', partition='2/frame', mirror='us', shards='shard=1,1000')
download_yt8m_shards(dataset_type='test', partition='2/frame', mirror='us', shards='shard=2,1000')
download_yt8m_shards(dataset_type='test', partition='2/frame', mirror='us', shards='shard=3,1000')


Starting download of test data... (Downloading shard=1,1000)


In [4]:
import tensorflow as tf

# Define feature description for parsing TFRecord files
feature_description = {
    'video_id': tf.io.FixedLenFeature([], tf.string),
    'labels': tf.io.VarLenFeature(tf.int64),  # Sparse, multi-labels
    'mean_rgb': tf.io.FixedLenFeature([1024], tf.float32, default_value=[0.0]*1024),  # RGB features
    'mean_audio': tf.io.FixedLenFeature([128], tf.float32, default_value=[0.0]*128)  # Audio features
}

# Parsing function for TFRecord files
def _parse_function(proto):
    print("Parsing record...")
    parsed_record = tf.io.parse_single_example(proto, feature_description)
    print(f"Parsed record: {parsed_record}")
    return parsed_record

# Extract features and labels from the parsed record
def extract_features_labels(record):
    features = {
        'mean_rgb': record['mean_rgb'],
        'mean_audio': record['mean_audio']
    }
    # If the label is missing, you can set a default label or skip it
    if 'labels' in record:
        labels = tf.sparse.to_dense(record['labels'], default_value=-1)
        print(f"Extracted labels: {labels}")
    else:
        labels = tf.constant([-1], dtype=tf.int64)  # Default label for missing cases
        print(f"Missing labels, assigning default: {labels}")
    return features, labels


# Function to load the dataset and limit to a few records for faster inspection
def load_dataset(file_pattern, batch_size=32, limit=None):
    """
    Load and preprocess the dataset.
    
    :param file_pattern: Path pattern for the dataset (e.g., '*.tfrecord')
    :param batch_size: Batch size for training
    :param limit: Limit number of records for testing, if None all data is used
    :return: TensorFlow dataset
    """
    print(f"Loading dataset from: {file_pattern}")
    raw_dataset = tf.data.TFRecordDataset(tf.io.gfile.glob(file_pattern))
    
    print("Parsing dataset...")
    parsed_dataset = raw_dataset.map(_parse_function)
    
    print("Extracting features and labels from dataset...")
    dataset = parsed_dataset.map(extract_features_labels)
    
    if limit:
        dataset = dataset.take(limit)  # Limit the dataset to a few records
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load datasets with a limit for faster experimentation
train_dataset = load_dataset('~/data/yt8m/frame/train/*.tfrecord', batch_size=32, limit=5)
validate_dataset = load_dataset('~/data/yt8m/frame/validate/*.tfrecord', batch_size=32, limit=5)
test_dataset = load_dataset('~/data/yt8m/frame/test/*.tfrecord', batch_size=32, limit=5)

# Inspect the first batch of training data
for features, labels in train_dataset.take(1):
    print("Mean RGB shape:", features['mean_rgb'].shape)
    print("Mean Audio shape:", features['mean_audio'].shape)
    print("Labels:", labels.numpy())



Loading dataset from: ~/data/yt8m/frame/train/*.tfrecord
Parsing dataset...
Parsing record...
Parsed record: {'labels': SparseTensor(indices=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:0", shape=(None, 1), dtype=int64), values=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:1", shape=(None,), dtype=int64), dense_shape=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:2", shape=(1,), dtype=int64)), 'mean_audio': <tf.Tensor 'ParseSingleExample/ParseExample/ParseExampleV2:3' shape=(128,) dtype=float32>, 'mean_rgb': <tf.Tensor 'ParseSingleExample/ParseExample/ParseExampleV2:4' shape=(1024,) dtype=float32>, 'video_id': <tf.Tensor 'ParseSingleExample/ParseExample/ParseExampleV2:5' shape=() dtype=string>}
Extracting features and labels from dataset...
Extracted labels: Tensor("SparseToDense:0", shape=(None,), dtype=int64)
Loading dataset from: ~/data/yt8m/frame/validate/*.tfrecord
Parsing dataset...
Parsing record...
Parsed record: {'labels': SparseTensor(indices=Tensor

In [11]:
import tensorflow as tf

# Function to read raw binary records from the TFRecord file
def inspect_shard_raw(file_path):
    dataset = tf.data.TFRecordDataset(file_path)
    for idx, raw_record in enumerate(dataset):
        try:
            print(f"Raw Record {idx}: {raw_record.numpy()[:100]}...")  # Print first 100 bytes of each record
        except Exception as e:
            print(f"Error reading record {idx}, skipping: {e}")

# Path to the shard you want to inspect
inspect_shard_raw("~/data/yt8m/frame/train/train0082.tfrecord")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 204: invalid continuation byte